In [ ]:
# Mounting Google Colab drive
from google.colab import drive
drive.mount('/content/drive')

# Imports
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, classification_report, recall_score, precision_score
from joblib import Parallel, delayed
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle

# Installing required packages
!pip install torch-geometric

import torch_geometric.transforms as T
from torch_geometric.nn import GATConv, Linear, to_hetero
from torch_geometric.data import HeteroData

# Change directory to location
loc = "/content/drive/MyDrive/KE_GNN/"
os.chdir(loc)
os.getcwd()

#number of epochs
epoch_n = 411
#sample number
run_number = 1

# move data to device
def move_to_device(obj, device):
  '''
  moves a dictionary to device (if needed)
  '''
  if isinstance(obj, torch.Tensor):
      return obj.to(device)
  elif isinstance(obj, dict):
      return {k: move_to_device(v, device) for k, v in obj.items()}
  elif isinstance(obj, list):
      return [move_to_device(i, device) for i in obj]
  elif isinstance(obj, tuple):
      return tuple(move_to_device(i, device) for i in obj)
  elif isinstance(obj, set):
      return {move_to_device(i, device) for i in obj}
  else:
      return obj

# empty dictionaries are being loaded in, inplace of the KE to reduce model modifications
# load train graph and clause dictionary
data_train = torch.load('{}Graph storage/post_hoc_train_graph.pt'.format(loc))
train_KE_location = {}

# load train graph and clause dictionary
data_valid = torch.load('{}Graph storage/post_hoc_valid_graph.pt'.format(loc))
valid_KE_location = {}

# load train graph and clause dictionary
data_test = torch.load('{}Graph storage/post_hoc_test_graph.pt'.format(loc))
test_KE_location = {}

# load knowledge enhancement
KE_conditions = {}

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')



# GAT model definition
class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        head = 2
        HC = hidden_channels * head
        # First graph attention layer
        self.conv1 = GATConv((-1, -1), hidden_channels, add_self_loops=False, heads=head)
        self.lin1 = Linear(-1, HC)

        # Second graph attention layer
        self.conv2 = GATConv((-1, -1), out_channels, add_self_loops=False)
        self.lin2 = Linear(-1, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index) + self.lin1(x)
        x = x.relu()
        x = self.conv2(x, edge_index) + self.lin2(x)
        return x

# Define the outer GNN model
class OuterGNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, metadata, KE_dictionary):
        super().__init__()
        self.gat = GAT(hidden_channels, out_channels)
        self.gat = to_hetero(self.gat, metadata, aggr='sum')
        self.lin = Linear(-1, out_channels)

    def forward(self, data, conditions):
        # loading data into GAT model
        x = self.gat(data.x_dict, data.edge_index_dict)
        # sigmoid out function
        x = torch.sigmoid(x['transaction'])

        rule_outputs = []
        KE_output_dic = {}
        return x, rule_outputs

# moving data to device if using GPU training
data_valid = data_valid.to(device)
data_test = data_test.to(device)
data_train = data_train.to(device)
metadata = data_train.metadata()

# moving clause location to device if using GPU training
KE_conditions = move_to_device(KE_conditions, device)
train_KE_location = move_to_device(train_KE_location, device)
valid_KE_location = move_to_device(valid_KE_location, device)
test_KE_location = move_to_device(test_KE_location, device)


# Instantiate the outer model
hidden_channels = 120
out_channels = 1
model = OuterGNN(hidden_channels, out_channels, metadata, KE_conditions)

criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model = model.to(device)

with torch.no_grad():  # Initialize lazy modules.
    out, _ = model(data_train, train_KE_location)


def inductive_train():
    '''
      Performs a single training step for the model.
    Returns:
        loss: The computed loss for the current training step.
        rule_outputs: The output from clause weights
    '''
    model.train()
    optimizer.zero_grad()  # Clear gradients

    out, rule_outputs = model(data_train, train_KE_location)  # Perform a single forward pass
    loss = criterion(out, data_train['transaction'].y)  # Compute the loss solely based on the training nodes
    loss.backward()
    optimizer.step()
    return loss, rule_outputs



def f1_finder(pred, true, max_val):
    '''
    Finds the best threshold for maximizing the F1-score of a binary classifier.

    Args:
        pred: Predicted values for the positive class.
        true: True binary labels.
        max_val: The maximum threshold value to consider.

    Returns:
        The threshold that maximizes the F1-score.
    '''
    thresholds = np.linspace(0, max_val, num=200, endpoint=True)

    def compute_f1(threshold):
        return f1_score(true, (pred > threshold).astype(int), zero_division=0.0)

    f1_scores = Parallel(n_jobs=-1)(delayed(compute_f1)(x) for x in thresholds)

    best_index = np.argmax(f1_scores)
    best_x = thresholds[best_index]
    return best_x


def test():
    '''
    Test the model on the validation set.

    Returns:
        The predicted and true labels for the validation set.
    '''
    model.eval()
    out, _ = model(data_valid, valid_KE_location)
    pred = out.detach().cpu().numpy()
    true_labels = data_valid['transaction'].y.cpu()
    return pred, true_labels.numpy()



f1_best = 0
prd = 0
best_model_state = None
measures = []
weights = []

for epoch in range(1, epoch_n):
  print(epoch)
  loss, rule_outputs = inductive_train()
  weights.append(rule_outputs.copy())
  if epoch % 10 == 0:
      print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
  if epoch % 10 == 0:
      pred, truess = test()

      threshold = f1_finder(pred, truess, 1.0)
      predss_thres = (pred > threshold).astype(int)

      f1 = f1_score(truess, predss_thres)
      print('Best current threshold:', threshold, 'Best F1 score:', f1, ' number of fraud: ', np.sum(truess))
      recall = recall_score(truess, predss_thres, zero_division = 0.0)
      prc = precision_score(truess, predss_thres, zero_division = 0.0)
      measures.append([epoch,loss,f1,threshold, recall,prc])

      if f1 > f1_best:
        print('new best model')
        f1_best = f1  # Update the best F1 score
        best_thresh = threshold

        torch.save({'epoch': epoch,'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss, }, '{}/model_storage/PH_GAT_{}.pt'.format(loc,run_number))
print('final best f1: {}, best threshold: {}'.format(pr_best, best_thresh))

# creating model
hidden_channels = 120
out_channels = 1
model = OuterGNN(hidden_channels, out_channels, metadata, KE_conditions)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


model = model.to(device)
checkpoint = torch.load('{}/model_storage/PH_GAT_{}.pt'.format(loc,run_number))


# Load the model and optimizer state dictionaries
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

# Set the model to evaluation mode

model.eval()

test_measure = []
out, _ = model(data_test, test_KE_location)
pred = out.detach().cpu().numpy()
true_labels = data_test['transaction'].y.cpu()
# predictions using the best threshold found in the best model
predss_thres = (pred > best_thresh).astype(int)
best_validation_threshold = best_thresh.copy()
# f1 score from the test
f1_val_thresh_test_set = f1_score(true_labels, predss_thres)
print('validation optimised test results:')
print(classification_report(true_labels, predss_thres))
recall_val_thres_test = recall_score(true_labels, predss_thres)
precision_val_thres_test = precision_score(true_labels, predss_thres)

# test optimised results
threshold = f1_finder(pred, true_labels, 1.0)
test_optimised_prediction = (pred > threshold).astype(int)

f1_test_optimised = f1_score(true_labels, test_optimised_prediction, zero_division = 0.0)
recall_test_optimised = recall_score(true_labels, test_optimised_prediction, zero_division = 0.0)
precision_test_optimised = precision_score(true_labels, test_optimised_prediction, zero_division = 0.0)

test_measure.append([f1_val_thresh_test_set, best_validation_threshold, recall_val_thres_test, precision_val_thres_test,
                   f1_test_optimised, threshold, recall_test_optimised, precision_test_optimised])

# final model results
df2 = pd.DataFrame(test_measure, columns = ['test_train_thresh_f1', 'test_train_thresh', 'test_train_recall', 'test_train_precision',
                                          'test_f1','test_thresh', 'test_recall', 'test_precision'])
#training results
df1 = pd.DataFrame(measures, columns=['epoch', 'training loss', 'optimised_f1','threshold', 'recall', 'precision'])


df1.to_csv('{}/Post Hoc/output/GAT/GAT_training_results_nest{}.csv'.format(loc,run_number))
df2.to_csv('{}/Post Hoc/output/GAT/GAT_test_results_nest{}.csv'.format(loc,run_number))

Mounted at /content/drive
Processing ./drive/MyDrive/torch_scatter-2.1.2+pt22cpu-cp310-cp310-linux_x86_64.whl
Processing ./drive/MyDrive/torch_sparse-0.6.18+pt22cpu-cp310-cp310-linux_x86_64.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 29.7 MB/s eta 0:00:00
1
2
3
4
5
6
7
8
9
10
Epoch: 010, Loss: 0.0107
Best current threshold: 0.0 Best F1 score: 0.002464221182472929  number of fraud:  3209.0
new best model
11
12
13
14
15
16
17
18
19
20
Epoch: 020, Loss: 0.0111
Best current threshold: 0.0 Best F1 score: 0.002464221182472929  number of fraud:  3209.0
final best f1: 0.002464221182472929, best threshold: 0.0
[[('RULE1', 1.0), ('RULE2', 1.0), ('RULE3', 1.0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00   2598314
         1.0       0.00      1.00      0.00      2951

    accuracy                           0.00   2601265
   macro avg       0.00      0.50      0.00   2601265
weighted avg       0.00      0.00      0.00   2601265



NameError: name 'out_data' is not defined